# Dropping Rows and Columns with Missing Values: Order Matters

#### by Fuat Akal


## Table of Content

[Problem](#problem)  
[Configurations](#Configurations)  
[Data Preparation](#data_preparation)   
[Evaluation](#evaluation)   
[Discussion](#discussion)   


## Problem <a class="anchor" id="problem"></a>

For various reasons, many real world datasets from the healthcare domain contain missing values, often encoded as blanks, NaNs or other placeholders (e.g., Not checked). Honestly, this is understandable. Because, such data is collected in a clinic environment, which is very busy and stressful. On the other hand, scikit-learn estimators assume that all values in a dataset ( I mean a numpy array) are numerical, and that all have and hold meaning. 

One simple strategy may be to get rid of entire rows/columns with missing values. Although I am a minimalist, this strategy is not a good one. It will very likely result in loosing precious data. 

Let us investigate!

## Configurations<a class="anchor" id="Configurations"></a>

In [9]:
# import libraries
import numpy as np
import pandas as pd
import random 

# configure constants
missingRows = 0.1
missingColumns = 0.25
numberOfRows = 100
numberOfColumns = 10
numPass = 3

#pd.options.display.max_columns = None
#pd.options.display.max_rows = None # default 60

## Data Preparation<a class="anchor" id="data_preparation"></a>

In [12]:
# Create a toy dataset to play with.
# It is a 100 by 10 matrix containing numbers between 0 and 9

data = {
    'a': np.random.randint(0, numberOfColumns, size=numberOfRows),
    'b': np.random.randint(0, numberOfColumns, size=numberOfRows),
    'c': np.random.randint(0, numberOfColumns, size=numberOfRows),
    'd': np.random.randint(0, numberOfColumns, size=numberOfRows),
    'e': np.random.randint(0, numberOfColumns, size=numberOfRows),
    'f': np.random.randint(0, numberOfColumns, size=numberOfRows),
    'g': np.random.randint(0, numberOfColumns, size=numberOfRows),
    'h': np.random.randint(0, numberOfColumns, size=numberOfRows),
    'i': np.random.randint(0, numberOfColumns, size=numberOfRows),
    'j': np.random.randint(0, numberOfColumns, size=numberOfRows),
}

# Create a dataframe
df = pd.DataFrame(data)

# Display top 5 rows
df.head()

,a,b,c,d,e,f,g,h,i,j
0,9,5,7,1,0,3,5,1,1,5
1,2,6,9,6,5,3,1,1,1,1
2,4,7,6,2,4,7,3,6,1,9
3,2,9,3,8,5,9,9,1,6,8
4,9,7,7,3,4,4,4,2,1,7


## Evaluation<a class="anchor" id="evaluation"></a>

In [180]:
# Let's check if there is any missing value in the dataset.
# There is no missing value in the dataframe yet.
# Soon, we will introduce some
df.isnull().sum()

a    0
b    0
c    0
d    0
e    0
f    0
g    0
h    0
i    0
j    0
dtype: int64

In [195]:
# We will introduce some missing values by simply replacing cells with np.nans.
# We iterate numPass times and create missing values for a random set of rows and columns 
for i in range(numPass):
    for col in df.sample(frac=missingColumns, axis='columns').columns:
        df.loc[df.sample(frac=missingRows).index, col] = np.nan

In [200]:
# Let's check if it worked
# Yes, it did. There are 27 missing value on column g.
df.isnull().sum()

a    10
b     0
c    10
d     0
e    10
f     0
g    27
h     0
i     0
j     0
dtype: int64

In [ ]:
# Save for future use
# df.to_csv("df.csv")

In [201]:
# Beware that your outputs may be different.
# Unless you execute the next line
# df = pd.read_csv("df.csv")

print("Removing all rows containing a missing value")
newRows, newColumns = df.dropna(axis=0).shape
print("Your dataframe has {} rows and {} columns now".format(newRows, newColumns))
print("You lost {}% of your samples.\n".format(np.round((1 - newRows/numberOfRows)*100),2))

print("Removing all columns containing a missing value")
newRows, newColumns = df.dropna(axis=1).shape
print("Your dataframe has {} rows and {} columns now".format(newRows, newColumns))
print("You lost {}% of your columns.\n".format(np.round((1 - newColumns/numberOfColumns)*100)))

print("Removing the columns with the most missing values and then, all rows containing a missing value")
newRows, newColumns = df.drop(['g'], axis=1).dropna(axis=0).shape
print("Your dataframe has {} rows and {} columns now".format(newRows, newColumns))
print("You lost {}% of your rows and {}% of your columns.\n".format(np.round((1 - newRows/numberOfRows)*100), np.round((1 - newColumns/numberOfColumns)*100)))

print("Removing two columns with high missing values and then, all rows containing a missing value")
newRows, newColumns = df.drop(['g', 'c'], axis=1).dropna(axis=0).shape
print("Your dataframe has {} rows and {} columns now".format(newRows, newColumns))
print("You lost {}% of your rows and {}% of your columns.\n".format(np.round((1 - newRows/numberOfRows)*100, 2), np.round((1 - newColumns/numberOfColumns)*100), 2))


Removing all rows containing a missing value
Your dataframe has 53 rows and 10 columns now
You lost 47.0% of your samples.

Removing all columns containing a missing value
Your dataframe has 100 rows and 6 columns now
You lost 40.0% of your columns.

Removing the columns with the most missing values and then, all rows containing a missing value
Your dataframe has 72 rows and 9 columns now
You lost 28.0% of your rows and 10.0% of your columns.

Removing two columns with high missing values and then, all rows containing a missing value
Your dataframe has 82 rows and 8 columns now
You lost 18.0% of your rows and 20.0% of your columns.



## Discussion <a class="anchor" id="discussion"></a>

Here we created some synthetic data with missing values. We tried some strategies for removing rows or columns with missing values.

1. Removing entire row or column with a missing value seems like the worst strategy. It resulted in the most data loss. We lost 47% of rows and 40% percent of columns with this strategy.

2. Removing columns with the highest missing value rates and then rows with a missing value. This strategy resulted in lower loss as compared to the first strategy. We lost 28% of rows and 10% percent of columns when we removed the column with the highest missing value rate and then rows with a missing value.

So, the second strategy seems to be working better. However, it is still a trade off. Your choice depends how many rows/columns you can tolerate to lose. Missing value counts on each column and each row must be inspected carefully. 



**Disclaimer!** This notebook is available for educational purposes only. There is no guarantee on the correctness of the content provided.

If you think there is any copyright violation, please let me [know](https://forms.gle/BNNRB2kR8ZHVEREq8). 
